In [25]:
import sys
import os

# 1. Add the specific directory where the .so file was found
module_path = '/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem'

if module_path not in sys.path:
    sys.path.append(module_path)

# 2. Verify files are visible to Python
print("Path added:", module_path)
print("Files in path:", [f for f in os.listdir(module_path) if "mass_spring" in f])

# 3. Import
import mass_spring
print("Success!")

Path added: /home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem
Files in path: ['mass_spring.cpython-312-x86_64-linux-gnu.so', 'test_mass_spring']
Success!


In [26]:
import sys
sys.path.append('/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [27]:
import math

mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# --- Configuration ---
num_bays = 10         # Longer arm for the crane
width = 1.0           # Width of the boom
height = 1.0          # Height of the boom
length = 1.0          # Length of each segment
stiffness = 25000     # Stiffer structure to hold the weight

cable_len = 3.0       # How long the cable hangs down
load_mass = 50.0      # The heavy weight at the end

# Calculate diagonal length (for bracing)
diag_len = math.sqrt(length**2 + height**2)
# Calculate cable diagonal (distance from top corners to the centered load)
cable_diag = math.sqrt((width/2)**2 + cable_len**2)

# --- 1. Create the Anchors (The "Tower" Connection) ---
# We fix the 4 corners at X=0
p_bl = mss.add(Fix((0, 0, 0)))          # Previous Bottom-Left
p_br = mss.add(Fix((0, width, 0)))      # Previous Bottom-Right
p_tr = mss.add(Fix((0, width, height))) # Previous Top-Right
p_tl = mss.add(Fix((0, 0, height)))     # Previous Top-Left

# --- 2. Build the Jib (The Arm) ---
for i in range(num_bays):
    x = (i + 1) * length
    
    # Standard truss segments (Lightweight structure)
    c_bl = mss.add(Mass(1.0, (x, 0, 0)))          # Current Bottom-Left
    c_br = mss.add(Mass(1.0, (x, width, 0)))      # Current Bottom-Right
    c_tr = mss.add(Mass(1.0, (x, width, height))) # Current Top-Right
    c_tl = mss.add(Mass(1.0, (x, 0, height)))     # Current Top-Left

    # --- A. Length Springs (The Long Lines) ---
    mss.add(Spring(length, stiffness, (p_bl, c_bl)))
    mss.add(Spring(length, stiffness, (p_br, c_br)))
    mss.add(Spring(length, stiffness, (p_tr, c_tr)))
    mss.add(Spring(length, stiffness, (p_tl, c_tl)))

    # --- B. Square Springs (The Cross Section Ring) ---
    mss.add(Spring(width,  stiffness, (c_bl, c_br))) 
    mss.add(Spring(height, stiffness, (c_br, c_tr))) 
    mss.add(Spring(width,  stiffness, (c_tr, c_tl))) 
    mss.add(Spring(height, stiffness, (c_tl, c_bl))) 

    # --- C. Diagonal Springs (The Strength) ---
    mss.add(Spring(diag_len, stiffness, (p_bl, c_tl))) 
    mss.add(Spring(diag_len, stiffness, (p_br, c_tr))) 
    mss.add(Spring(diag_len, stiffness, (p_tl, c_tr))) 
    mss.add(Spring(diag_len, stiffness, (p_bl, c_br))) 

    # Update 'previous' nodes to be the 'current' ones for the next loop
    p_bl, p_br, p_tr, p_tl = c_bl, c_br, c_tr, c_tl

# --- 3. The Crane Mechanism (Cable & Load) ---
# At this point, p_tr and p_tl are the nodes at the very tip of the crane (top).

# Calculate position: End of X, Centered in Y, Hanging down from Top Z
tip_x = num_bays * length
load_y = width / 2.0
load_z = height - cable_len

# Create the Heavy Load
load_node = mss.add(Mass(load_mass, (tip_x, load_y, load_z)))



# Connect the Load (The Cable)
# We attach the load to the top two corners of the crane tip
mss.add(Spring(cable_diag, stiffness, (p_tl, load_node)))
mss.add(Spring(cable_diag, stiffness, (p_tr, load_node)))

121

In [28]:
# #for a 2 mass pendulum
# mss = MassSpringSystem3d()
# mss.gravity = (0,0,-9.81)

# mA = mss.add (Mass(1, (1,0,0)))
# mB = mss.add (Mass(2, (2,0,0)))
# f1 = mss.add (Fix( (0,0,0)) )
# mss.add (Spring(1, 200000, (f1, mA)))
# mss.add (Spring(1, 100000, (mA, mB)))

In [29]:
# #To test for a chain of elements
# mss = MassSpringSystem3d()
# mss.gravity = (0,0,-9.81)


# number_of_masses = 10
# starting_position = (0,0,0)
# length_chain = 9
# single_length = length_chain/number_of_masses

# prev_obj = mss.add(Fix(starting_position))#to store the previous object for the springs
# for i in range(number_of_masses):
#     pos = (starting_position[0] + (i + 1) * single_length, starting_position[1], starting_position[2])
#     new_mass = mss.add(Mass(1.0, pos)) #adding new masses
#     mss.add(Spring(1, 10000, (prev_obj, new_mass)))
#     prev_obj = new_mass
    

In [30]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [31]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [ ]:
from time import sleep
for i in range(1000):
    mss.simulate(0.005, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)